## hand representation
- bit-vector

In [1]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Params

In [3]:
_RUN_UNIT_TESTS = True

## Game constants

In [4]:
# game constants
NUM_RANKS = 13
NUM_SUITS = 4
NUM_CARDS = NUM_RANKS * NUM_SUITS
CARDS = [(s, r) for s in range(NUM_SUITS) for r in range(NUM_RANKS)]

STRAIGHT_FLUSH_LEN = 5
STRAIGHT_LEN = 5
FLUSH_LEN = 5
FOUR_KIND_LEN = 4
THREE_KIND_LEN = 3
TWO_KIND_LEN = 2


In [5]:
# ratings
RATING_OFFSET = 2 * NUM_RANKS

STRAIGHT_FLUSH_RATING = 9 << RATING_OFFSET
FOUR_KIND_RATING = 8 << RATING_OFFSET
FULL_HOUSE_RATING = 7 << RATING_OFFSET
FLUSH_RATING = 6 << RATING_OFFSET
STRAIGHT_RATING = 5 << RATING_OFFSET
THREE_KIND_RATING = 4 << RATING_OFFSET
TWO_PAIR_RATING = 3 << RATING_OFFSET
TWO_KIND_RATING = 2 << RATING_OFFSET
ONE_KIND_RATING = 1 << RATING_OFFSET


RATING_TYPE_MASK = 15 << RATING_OFFSET
RATING_KICKER2_MASK = (1 << NUM_RANKS) - 1
RATING_KICKER1_MASK = RATING_KICKER2_MASK << NUM_RANKS


## Masks
- _suit_mask: _suit_mask[s] gives bit mask for all cards of suit s
- _bit_mask: _bit_mask[ind] gives bit mask for bitind 'ind'
- _bit_mask2: _bit_mask2[s][r] gives bit mask for card (s, r)

In [6]:
_all_mask = (1 << NUM_CARDS) - 1
_bit_mask = [1 << i for i in range(NUM_CARDS)]
_bit_mask2 = [[(1 << (s * NUM_RANKS + r)) for r in range(NUM_RANKS)] for s in range(NUM_SUITS)]

_bit_unmask = {m: i for i, m in enumerate(_bit_mask)}
_bit_unmask[0] = -1

In [7]:
_suit_masks = [((1 << NUM_RANKS) - 1) << (s * NUM_RANKS) for s in range(NUM_SUITS)]
_rank_masks = [sum([1 << (s * NUM_RANKS + r) for s in range(NUM_SUITS)]) for r in range(NUM_RANKS)]

_suit_mask = _suit_masks[0]

In [8]:
base_masks = []
for s1 in range(NUM_SUITS):
    for s2 in range(s1):
        val = _bit_mask[s1 * NUM_RANKS] + _bit_mask[s2 * NUM_RANKS]
        base_masks += [val]

_2k_masks = []
for r in range(NUM_RANKS):
    _2k_masks += [bmask << r for bmask in base_masks]

## Utils

In [9]:
def binify(bits_list):
    return [bin(b) for b in bits_list]

In [10]:
def run_unit_tests(unit_fn):
    if _RUN_UNIT_TESTS:
        unit_fn()
        print(f"`{unit_fn.__name__}` success!")
    else:
        print(f"Skipping `{unit_fn.__name__}`")


In [40]:
# rating utils
def rank_bits_to_nums(ranks):
    vals = []
    for r in range(NUM_RANKS):
        if (_bit_mask[r] & ranks) > 0:
            vals += [r]
    return tuple(reversed(vals))

def rating_get_type(rating):
    return (rating & RATING_TYPE_MASK) >> (2 * NUM_RANKS)

def rating_get_kicker1(rating):
    return rank_bits_to_nums((rating & RATING_KICKER1_MASK) >> NUM_RANKS)

def rating_get_kicker2(rating):
    return rank_bits_to_nums((rating & RATING_KICKER2_MASK))

def rating_bits_to_tuple(rating):
    return (rating_get_type(rating), rating_get_kicker1(rating), rating_get_kicker2(rating))


_rating_type_to_str = [None, "highcard", "pair", "twopair", "threes", "straight", "flush", "fullhouse", "fours", "straightflush"]

def rating_prettify(rating):
    (rtype, k1, k2) = rating_bits_to_tuple(rating)
    
    return (_rating_type_to_str[rtype], k1, k2)

In [41]:
def inv_bits(hand):
    return hand ^ _all_mask

def rbits_to_bits(rbits):
    return rbits + (rbits << NUM_RANKS) + (rbits << 2 * NUM_RANKS) + (rbits << 3 * NUM_RANKS)

def remove_ranks(hand, rbits):
    """ Removes all ranks in rbits from bits
    bits - original bitvec
    rbits - bit ranks to remove
    """
    keep_bits = inv_bits(rbits_to_bits(rbits))
    return hand & keep_bits

def get_bits_by_suit(hand):
    vals = []
    for s in range(NUM_SUITS):
        vals += [(hand >> (s * NUM_RANKS)) & _suit_mask]
    return vals

In [42]:
# returns highest bit (1 << bitind)

def get_n_highest_bits(bits, n):
    v = 0
    for i in range(n):
        b = get_highest_bit(bits)
        bits -= b
        v += b
    return v

def get_highest_bit(bits):
    # ensure it has at least one bit
    if bits == 0:
        return 0
    
    return _bit_mask[bits.bit_length() - 1]



In [43]:
# returns bit vector of NUM_RANKS length
def get_1k_ranks(hand):
    bits_by_suit = get_bits_by_suit(hand)
    b1, b2, b3, b4 = bits_by_suit
    return b1 | b2 | b3 | b4
    
#     total = 0
#     for s in range(NUM_SUITS):
#         total = total | (bits >> (s * NUM_RANKS))
#     return total & _suit_mask

def get_highest_1k(hand, n=1):
    ranks = get_1k_ranks(hand)
    return get_highest_bit(ranks)

def get_n_highest_1k(hand, n):
    ranks = get_1k_ranks(hand)
    return get_n_highest_bits(ranks, n)



def get_2k_ranks(hand):
    bits_by_suit = get_bits_by_suit(hand)
    b1, b2, b3, b4 = bits_by_suit
    return (b1 & b2) | (b1 & b3) | (b1 & b4) | (b2 & b3) | (b2 & b4) | (b3 & b4)
    
def get_highest_2k(hand):
    ranks = get_2k_ranks(hand)
    return get_highest_bit(ranks)

def get_n_highest_2k(hand, n):
    ranks = get_2k_ranks(hand)
    return get_n_highest_bits(ranks, n)



def get_3k_ranks(bits):
    bits_by_suit = get_bits_by_suit(bits)
    b1, b2, b3, b4 = bits_by_suit
    return (b1 & b2 & b3) | (b1 & b2 & b4) | (b1 & b3 & b4) | (b2 & b3 & b4)
    
def get_highest_3k(hand):
    ranks = get_3k_ranks(hand)
    return get_highest_bit(ranks)

def get_n_highest_3k(hand, n):
    ranks = get_3k_ranks(bits)
    return get_n_highest_bits(ranks, n)



def get_4k_ranks(hand):
    bits_by_suit = get_bits_by_suit(hand)
    b1, b2, b3, b4 = bits_by_suit
    return b1 & b2 & b3 & b4
    
def get_highest_4k(hand):
    ranks = get_4k_ranks(hand)
    return get_highest_bit(ranks)

def get_n_highest_4k(hand, n):
    ranks = get_4k_ranks(hand)
    return get_n_highest_bits(ranks, n)

In [44]:
## cards are a tuple representation of a hand (bit-vector)

def validate_card(s, r):
    assert 0 <= s < NUM_SUITS and 0 <= r < NUM_RANKS

def validate_cards(cards):
    assert len(set(cards)) == len(cards) # ensure cards are unique
    for s, r in cards:
        validate_card(s, r)

def validate_ind(ind):
    assert 0 <= ind < NUM_CARDS

def card_to_ind(s, r):
    validate_card(s, r)
    return s * NUM_RANKS + r

def ind_to_card(ind):
    validate_ind(ind)
    return (ind // NUM_RANKS, ind % NUM_RANKS)

def hand_to_cards(hand):
    assert 0 <= hand < (1 << NUM_CARDS)
    
    cards = []
    for ind, card in enumerate(CARDS):
        if (hand & _bit_mask[ind]) > 0:
            cards += [card]
            
    return cards

def cards_to_hand(cards):
    validate_cards(cards)
    
    hand = 0
    for s, r in cards:
        hand += _bit_mask[card_to_ind(s, r)]
    return hand

            
def unit_tests__card_to_ind():
    assert ( card_to_ind(0, 0) == 0 )
    assert ( card_to_ind(0, 1) == 1 )
    assert ( card_to_ind(1, 1) == 14 )
    assert ( card_to_ind(2, 6) == 32 )
    assert ( card_to_ind(3, 12) == 51 )
    
def unit_tests__ind_to_card():
    assert ( ind_to_card(0) == (0, 0) )
    assert ( ind_to_card(5) == (0, 5) )
    assert ( ind_to_card(13) == (1, 0) )
    assert ( ind_to_card(51) == (3, 12) )
    

def unit_tests__hand_to_cards():
    assert ( tuple(hand_to_cards(0b11)) == ((0, 0), (0, 1)) )
    assert ( tuple(hand_to_cards((1 << 13) + 7)) == ((0, 0), (0, 1), (0, 2), (1, 0)) )
    assert ( tuple(hand_to_cards((3 << 13) + 7)) == ((0, 0), (0, 1), (0, 2), (1, 0), (1, 1)) )


run_unit_tests(unit_tests__card_to_ind)
run_unit_tests(unit_tests__ind_to_card)
run_unit_tests(unit_tests__hand_to_cards)
    

`unit_tests__card_to_ind` success!
`unit_tests__ind_to_card` success!
`unit_tests__hand_to_cards` success!


## ratings

- straight flush (SF)
    - highest value (1 number, 13 bits)

- four of a kind (4K)
    - four + kicker (1 +  numbers, 13 bits)

- full house (FH)
    - triple + pair (1 + 1 numbers, 26 bits)

- flush (FL)
    - highest five numbers (5 numbers, 13 bits)
  
- straght (ST)
    - highest value 1 number, 13 bits)

- triple (3K)
    - triple + kicker1 + kicker2 (1 + 2 highest numbers, 26 bits)
    
- two pair (2P)
    - pair1 + pair2 + kicker (2 + 1 highest number, 26 bits)

- pair (2K)
    - pair + kicker1 + kicker2 + kicker3 (1 + 3 highest numbers, 13 bits)

- high card (1K)
    - highest five numbers (5 nums, 13 bits)

In [45]:
# check for a straight flush
base_mask = (1 << STRAIGHT_FLUSH_LEN) - 1
_sf_masks = [base_mask << (r + s * NUM_RANKS) for r in range(NUM_RANKS - STRAIGHT_FLUSH_LEN + 1) for s in range(NUM_SUITS)]

# use same base mask as _sf_masks
_straight_masks = [base_mask << r for r in range(NUM_RANKS - STRAIGHT_LEN + 1)]

In [46]:
def check_straight_flush(n):
    is_sf_mask = (1 << STRAIGHT_FLUSH_LEN) - 1

    rating = 0

    # since this goes from small to big, we can use `rating = val`
    for mask in _sf_masks:
        masked = mask & n
        
        ind = (mask.bit_length() - 1) // NUM_RANKS * NUM_RANKS
        
        if masked.bit_count() == STRAIGHT_FLUSH_LEN:
            rating = masked >> ind

    if rating > 0:
        return STRAIGHT_FLUSH_RATING + rating
    else:
        return 0
    
def unit_tests__check_straight_flush():
    assert check_straight_flush(0b11111) == STRAIGHT_FLUSH_RATING + 0b11111
    assert check_straight_flush(0b11111 << 13) == STRAIGHT_FLUSH_RATING + 0b11111
    assert check_straight_flush(0b11111 << 12) == 0
    assert check_straight_flush(0b11111 << 9) == 0
    assert check_straight_flush(0b11111 << 8) == STRAIGHT_FLUSH_RATING + (0b11111 << 8)
    assert check_straight_flush(0b11111 << 39) == STRAIGHT_FLUSH_RATING + (0b11111)
    assert check_straight_flush(0b11111 << 47) == STRAIGHT_FLUSH_RATING + (0b11111 << 8)
    
    assert check_straight_flush(0b1111111) == STRAIGHT_FLUSH_RATING + (0b11111 << 2)
    assert check_straight_flush(0b111111 << 8) == STRAIGHT_FLUSH_RATING + (0b11111 << 8)
    assert check_straight_flush(0b11111 << 12) == 0
    assert check_straight_flush(0b111111 << 12) == STRAIGHT_FLUSH_RATING + (0b11111)
    

    
run_unit_tests(unit_tests__check_straight_flush)

`unit_tests__check_straight_flush` success!


In [47]:
def check_four_kind(bits):

    fours = get_highest_4k(bits)

    if fours > 0:
        # find kicker
        bits = remove_ranks(bits, fours)
        kicker = get_highest_1k(bits)

        return FOUR_KIND_RATING + (fours << NUM_RANKS) + kicker
    else:
        return 0
    
def unit_tests__check_four_kind():

    base_bits = (1 << 0) + (1 << 13) + (1 << 26) + (1 << 39)

    bits = base_bits + (base_bits << 10) + (1 << 3)
    ans = FOUR_KIND_RATING + (1 << 10 << NUM_RANKS) + (1 << 3)
    assert check_four_kind(bits) == ans
    
    bits = (base_bits - 1)
    ans = 0
    assert check_four_kind(bits) == ans

    bits = (base_bits << 5) + (base_bits << 10) + (1 << 3)
    ans = FOUR_KIND_RATING + (1 << 10 << NUM_RANKS) + (1 << 5)
    assert check_four_kind(bits) == ans
    
    bits = (1 << NUM_CARDS) - 1
    ans = FOUR_KIND_RATING + (1 << (NUM_RANKS - 1) << NUM_RANKS) + (1 << (NUM_RANKS - 2))
    assert check_four_kind(bits) == ans
    
    bits = (1 << NUM_CARDS) - 1
    ans = FOUR_KIND_RATING + (1 << (NUM_RANKS - 1) << NUM_RANKS) + (1 << (NUM_RANKS - 2))
    assert check_four_kind(bits) == ans
    
run_unit_tests(unit_tests__check_four_kind)

`unit_tests__check_four_kind` success!


In [48]:
def check_full_house(bits):
    triple = get_highest_3k(bits)
    if triple > 0:
        bits = remove_ranks(bits, triple)
        double = get_highest_2k(bits)
        if double > 0:
            return FULL_HOUSE_RATING + (triple << NUM_RANKS) + double
    
    return 0


def unit_tests__check_full_house():

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2)])
    ans = FULL_HOUSE_RATING + (1 << 1 << NUM_RANKS) + (1 << 2)
    print(rating_bits_to_tuple(check_full_house(bits)))
    assert check_full_house(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2)])
    ans = FULL_HOUSE_RATING + (1 << 2 << NUM_RANKS) + (1 << 1)
    print(rating_bits_to_tuple(check_full_house(bits)))
    assert check_full_house(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (0, 2), (1, 2)])
    ans = 0
    print(rating_bits_to_tuple(check_full_house(bits)))
    assert check_full_house(bits) == ans
    
    
run_unit_tests(unit_tests__check_full_house)

(7, (1,), (2,))
(7, (2,), (1,))
(0, (), ())
`unit_tests__check_full_house` success!


In [49]:
def check_flush(bits):
    bits_by_suit = get_bits_by_suit(bits)
    
    rating = 0
    for rbits in bits_by_suit:
        masked = get_n_highest_bits(rbits, n=5)
        if masked.bit_count() == FLUSH_LEN:
            rating = max(rating, masked)

    if rating > 0:
        return FLUSH_RATING + rating
    else:
        return 0

    
def unit_tests__check_flush():

    bits = cards_to_hand([(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 6), (0, 7)])
    ans = FLUSH_RATING + (0b10111100)
    val = check_flush(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5), (0, 7), (0, 9), (3, 11)])
    ans = FLUSH_RATING + _bit_mask[1] + _bit_mask[2] + _bit_mask[5] + _bit_mask[7] + _bit_mask[9]
    val = check_flush(bits)
    assert val == ans

    bits = cards_to_hand([(3, 0), (0, 1), (1, 2), (2, 3), (3, 4)])
    ans = 0
    val = check_flush(bits)
    assert val == ans

    bits = cards_to_hand([(3, 0), (0, 1), (1, 2), (2, 3), (3, 4), (3, 8), (0, 9), (1, 10), (2, 11), (3, 12)])
    ans = 0
    val = check_flush(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (0, 10)])
    ans = 0
    val = check_flush(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (0, 2), (1, 2)])
    ans = 0
    val = check_flush(bits)
    assert val == ans
    

run_unit_tests(unit_tests__check_flush)

`unit_tests__check_flush` success!


In [50]:
def check_straight(bits):
    rbits = get_1k_ranks(bits)
    
    rating = 0
    
    # since this goes from small to big, we can use `rating = val`
    for mask in _straight_masks:
        masked = rbits & mask
        if masked.bit_count() == STRAIGHT_LEN:
            rating = masked

    if rating > 0:
        return STRAIGHT_RATING + rating
    else:
        return 0

    
def unit_tests__check_straight():

    bits = cards_to_hand([(0, 1), (1, 2), (2, 3), (3, 4), (0, 5), (1, 6), (0, 7)])
    ans = STRAIGHT_RATING + (0b11111 << 3)
    val = check_straight(bits)
    assert val == ans

    bits = cards_to_hand([(3, 0), (0, 1), (1, 2), (2, 3), (3, 4)])
    ans = STRAIGHT_RATING + (0b11111 << 0)
    val = check_straight(bits)
    assert val == ans

    bits = cards_to_hand([(3, 0), (0, 1), (1, 2), (2, 3), (3, 4), (3, 8), (0, 9), (1, 10), (2, 11), (3, 12)])
    ans = STRAIGHT_RATING + (0b11111 << 8)
    val = check_straight(bits)
    assert val == ans

    

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5), (0, 7), (0, 9), (3, 11)])
    ans = 0
    val = check_straight(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (0, 10)])
    ans = 0
    val = check_straight(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (0, 2), (1, 2)])
    ans = 0
    val = check_straight(bits)
    assert val == ans
    

run_unit_tests(unit_tests__check_straight)

`unit_tests__check_straight` success!


In [51]:
def check_three_kind(bits):
    triple = get_highest_3k(bits)
    if triple > 0:
        bits = remove_ranks(bits, triple)
        kickers = get_n_highest_1k(bits, n=2)
        return THREE_KIND_RATING + (triple << NUM_RANKS) + kickers
    
    return 0


def unit_tests__check_three_kind():

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5)])
    ans = THREE_KIND_RATING + (1 << 1 << NUM_RANKS) + (1 << 2) + (1 << 5)
#     print(rating_bits_to_tuple(check_three_kind(bits)))
    assert check_three_kind(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (0, 10)])
    ans = THREE_KIND_RATING + (1 << 2 << NUM_RANKS) + (1 << 1) + (1 << 10)
#     print(rating_bits_to_tuple(check_three_kind(bits)))
    assert check_three_kind(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (0, 2), (1, 2)])
    ans = 0
#     print(rating_bits_to_tuple(check_three_kind(bits)))
    assert check_three_kind(bits) == ans
    

run_unit_tests(unit_tests__check_three_kind)

`unit_tests__check_three_kind` success!


In [52]:
def check_two_pair(bits):
    doubles = get_n_highest_2k(bits, n=2)
    if doubles.bit_count() >= 2:
        bits = remove_ranks(bits, doubles)
        kickers = get_highest_1k(bits)
        return TWO_PAIR_RATING + (doubles << NUM_RANKS) + kickers
    
    return 0


def unit_tests__check_two_pair():

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5)])
    ans = TWO_PAIR_RATING + ((_bit_mask[2] + _bit_mask[1]) << NUM_RANKS) + _bit_mask[5]
    print(rating_bits_to_tuple(check_two_pair(bits)))
    assert check_two_pair(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5), (0, 7), (0, 9)])
    ans = TWO_PAIR_RATING + ((_bit_mask[2] + _bit_mask[1]) << NUM_RANKS) + _bit_mask[9]
    print(rating_bits_to_tuple(check_two_pair(bits)))
    assert check_two_pair(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (0, 10)])
    ans = TWO_PAIR_RATING + ((_bit_mask[2] + _bit_mask[1]) << NUM_RANKS) + _bit_mask[10]
    print(rating_bits_to_tuple(check_two_pair(bits)))
    assert check_two_pair(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (0, 2), (1, 2)])
    ans = TWO_PAIR_RATING + ((_bit_mask[2] + _bit_mask[1]) << NUM_RANKS)
    print(rating_bits_to_tuple(check_two_pair(bits)))
    assert check_two_pair(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 2)])
    ans = 0
    print(rating_bits_to_tuple(check_two_pair(bits)))
    assert check_two_pair(bits) == ans
    

run_unit_tests(unit_tests__check_two_pair)

(3, (2, 1), (5,))
(3, (2, 1), (9,))
(3, (2, 1), (10,))
(3, (2, 1), ())
(0, (), ())
`unit_tests__check_two_pair` success!


In [53]:
def check_two_kind(bits):
    double = get_highest_2k(bits)
    if double > 0:
        bits = remove_ranks(bits, double)
        kickers = get_n_highest_1k(bits, n=3)
        return TWO_KIND_RATING + (double << NUM_RANKS) + kickers
    
    return 0


def unit_tests__check_two_kind():

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5)])
    ans = TWO_KIND_RATING + (1 << 2 << NUM_RANKS) + (1 << 1) + (1 << 5)
#     print(rating_bits_to_tuple(check_two_kind(bits)))
    assert check_two_kind(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5), (0, 7), (0, 9)])
    ans = TWO_KIND_RATING + (1 << 2 << NUM_RANKS) + (1 << 9) + (1 << 7) + (1 << 5)
#     print(rating_bits_to_tuple(check_two_kind(bits)))
    assert check_two_kind(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (0, 10)])
    ans = TWO_KIND_RATING + (1 << 2 << NUM_RANKS) + (1 << 1) + (1 << 10)
#     print(rating_bits_to_tuple(check_two_kind(bits)))
    assert check_two_kind(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 1), (0, 2), (1, 2)])
    ans = TWO_KIND_RATING + (1 << 2 << NUM_RANKS) + (1 << 1)
#     print(rating_bits_to_tuple(check_two_kind(bits)))
    assert check_two_kind(bits) == ans

    bits = cards_to_hand([(0, 1), (1, 2)])
    ans = 0
#     print(rating_bits_to_tuple(check_two_kind(bits)))
    assert check_two_kind(bits) == ans
    

run_unit_tests(unit_tests__check_two_kind)

`unit_tests__check_two_kind` success!


In [54]:
def check_one_kind(bits):
    kickers = get_n_highest_1k(bits, n=5)
    return ONE_KIND_RATING + kickers

def unit_tests__check_one_kind():

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5)])
    ans = ONE_KIND_RATING + _bit_mask[1] + _bit_mask[2] + _bit_mask[5]
    val = check_one_kind(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5), (0, 7), (0, 9)])
    ans = ONE_KIND_RATING + _bit_mask[1] + _bit_mask[2] + _bit_mask[5] + _bit_mask[7] + _bit_mask[9]
    val = check_one_kind(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (0, 5), (0, 7), (0, 9), (3, 11)])
    ans = ONE_KIND_RATING + _bit_mask[2] + _bit_mask[5] + _bit_mask[7] + _bit_mask[9] + _bit_mask[11]
    val = check_one_kind(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (2, 1), (3, 1), (0, 2), (1, 2), (2, 2), (0, 10)])
    ans = ONE_KIND_RATING + _bit_mask[1] + _bit_mask[2] + _bit_mask[10]
    val = check_one_kind(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 1), (0, 2), (1, 2)])
    ans = ONE_KIND_RATING + _bit_mask[1] + _bit_mask[2]
    val = check_one_kind(bits)
    assert val == ans

    bits = cards_to_hand([(0, 1), (1, 2)])
    ans = ONE_KIND_RATING + _bit_mask[1] + _bit_mask[2]
    val = check_one_kind(bits)
    assert val == ans

    bits = cards_to_hand([])
    ans = ONE_KIND_RATING
    val = check_one_kind(bits)
    assert val == ans
    

run_unit_tests(unit_tests__check_one_kind)

`unit_tests__check_one_kind` success!


In [55]:
def get_rating(hand):
    vals = [
        check_straight_flush(hand),
        check_four_kind(hand),
        check_full_house(hand),
        check_flush(hand),
        check_straight(hand),
        check_three_kind(hand),
        check_two_pair(hand),
        check_two_kind(hand),
        check_one_kind(hand),
    ]
    
    return max(vals)
    

In [61]:
rating = get_rating(cards_to_hand([(0, 1), (0, 2), (3, 12), (2, 12)]))
rating_prettify(rating)

('pair', (12,), (2, 1))